In [107]:
import numpy as np
import pandas as pd

raw_train_data = pd.read_csv('covid_train.csv')
raw_valid_data = pd.read_csv('covid_valid.csv')

In [85]:
def preprocessing(data):
  data['age_range'] = pd.cut(x=data['age'], bins=[0,10,20,30,40,50,60,70,80,90,100,120], labels=['0s','10s','20s','30s','40s','50s','60s','70s','80s','90s','100s']) 
  new_data = data.drop(['entry_date', 'date_symptoms','age'], axis=1)
  for col_name in new_data.columns:
    new_data[col_name].replace(99, 'unknown',inplace=True)
    new_data[col_name].replace(98, 'unknown',inplace=True)
    new_data[col_name].replace(97, 'unknown',inplace=True)
  return new_data

In [122]:
def NB(raw_data):
  data = preprocessing(raw_data)

  prior_p = np.zeros(2)
  prior_p[0] = sum(data['date_died']=='9999-99-99')/len(data['date_died'])
  prior_p[1] = 1 - prior_p[0]

  x = data.drop(['date_died'], axis=1)
  cd = {}
  for col_name in x.columns:
    feature_cd = np.zeros((2,len(set(x[col_name]))))
    for n in range(len(list(set(x[col_name])))):
      feature_n_surive = x.loc[(x[col_name]==list(set(x[col_name]))[n])&(data['date_died']=='9999-99-99'),].shape[0]
      feature_cd[0,n] = (feature_n_surive+1) / sum(data['date_died']=='9999-99-99')
      feature_n_died = x.loc[(x[col_name]==list(set(x[col_name]))[n])&(data['date_died']!='9999-99-99'),].shape[0]
      feature_cd[1,n] = (feature_n_died+1) / sum(data['date_died']!='9999-99-99')

    feature_cd = pd.DataFrame(feature_cd,columns=list(set(x[col_name])),index=[0,1])
    cd[col_name] = feature_cd

  return prior_p, cd

prior_p, cd = NB(raw_train_data)

In [143]:
# cd['icu'].iloc[0,:]
data = preprocessing(raw_train_data)
data.loc[1,"sex"]

2

In [146]:
def predict(raw_test_data):
  test_data = preprocessing(raw_test_data)
  x_test = data.drop(['date_died'], axis=1)
  result = []
  for row in range(x_test.shape[0]):
    p0 = prior_p[0]
    p1 = prior_p[1]
    for col_name in x_test.columns:
      p0 = p0*cd[col_name].iloc[0,:][x_test.loc[row,col_name]]
      p1 = p1*cd[col_name].iloc[1,:][x_test.loc[row,col_name]]
    
  if p0 > p1:
    result.append(0)
  else:
    result.append(1)

  return result,accuracy

In [147]:
result = predict(raw_valid_data)

KeyboardInterrupt: ignored